In [3]:
import pandas as pd
from sklearn import svm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models

# Load data from the txt file
data = pd.read_csv('comparing_dl_vs_non_dl/non_linear_classification.csv', header=None)


In [2]:
x = data[[0, 1]]
y = data[2]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [78]:
# A neural network with multiple hidden layers
deepneural_1 = models.Sequential([
    # input layer
    layers.Dense(64, activation='relu', input_shape=(2,)),
    # multiple hidden layers
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    # output layer with one neuron
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
deepneural_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
deepneural_1.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 64)                192       
                                                                 
 dense_49 (Dense)            (None, 64)                4160      
                                                                 
 dense_50 (Dense)            (None, 64)                4160      
                                                                 
 dense_51 (Dense)            (None, 64)                4160      
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 64)                4160      
                                                                 
 dense_54 (Dense)            (None, 64)               

In [79]:

deepneural_1_hist = deepneural_1.fit(x_train, y_train, epochs=10, batch_size=10)




Epoch 1/10
201/201 [==============================] - 2s 2ms/step - loss: 0.4977 - accuracy: 0.7290
Epoch 2/10
201/201 [==============================] - 0s 2ms/step - loss: 0.3342 - accuracy: 0.8493
Epoch 3/10
201/201 [==============================] - 0s 2ms/step - loss: 0.2818 - accuracy: 0.8847
Epoch 4/10
201/201 [==============================] - 0s 2ms/step - loss: 0.2549 - accuracy: 0.9057
Epoch 5/10
201/201 [==============================] - 0s 2ms/step - loss: 0.2446 - accuracy: 0.9052
Epoch 6/10
201/201 [==============================] - 0s 1ms/step - loss: 0.2468 - accuracy: 0.9022
Epoch 7/10
201/201 [==============================] - 0s 1ms/step - loss: 0.2238 - accuracy: 0.9157
Epoch 8/10
201/201 [==============================] - 0s 2ms/step - loss: 0.2106 - accuracy: 0.9097
Epoch 9/10
201/201 [==============================] - 0s 2ms/step - loss: 0.1820 - accuracy: 0.9182
Epoch 10/10
201/201 [==============================] - 0s 1ms/step - loss: 0.2028 - accuracy: 0.9092

In [80]:
converter = tf.lite.TFLiteConverter.from_keras_model(deepneural_1)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/37/z4jydb4976915t7k69vbvb4c0000gn/T/tmpi7qwtj14/assets


INFO:tensorflow:Assets written to: /var/folders/37/z4jydb4976915t7k69vbvb4c0000gn/T/tmpi7qwtj14/assets
2023-01-23 00:09:14.585915: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-23 00:09:14.585933: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-23 00:09:14.586134: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/37/z4jydb4976915t7k69vbvb4c0000gn/T/tmpi7qwtj14
2023-01-23 00:09:14.588946: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-23 00:09:14.588964: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/37/z4jydb4976915t7k69vbvb4c0000gn/T/tmpi7qwtj14
2023-01-23 00:09:14.600981: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-23 00:09:14.709334: I tensorflow/cc/saved_model/loader.cc:213] Running initialization

In [81]:
with open('lite.tflite', 'wb') as f:
      f.write(tflite_model)

In [83]:
import numpy as np
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="lite.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create test input data
input_data = np.array([0.50873,0.08490], dtype=np.float32).reshape(1,2)

# Run the model on the test input data
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get the model's output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output
print(output_data)


# Create test input data
input_data = np.array([0.48690,0.58265], dtype=np.float32).reshape(1,2)

# Run the model on the test input data
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# Get the model's output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the output
print(output_data)


[[0.99997276]]
[[0.1151121]]


As we can see in the above code the saved model can now be reused. It seems to make the right predictions, so we can conclude that it works. 